In [8]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/PC3_Termination3/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import make_grid
from torch.autograd import grad as torch_grad
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
import numpy as np
import imageio
import time
import pickle
from joblib import Parallel, delayed
import joblib
import os

In [10]:
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

# Number of channels in the training images. For color images this is 3
nc = 1
# Size of z latent vector (i.e. size of generator input)
nz = 100
# Size of feature maps in generator
ngf = 64
# Size of feature maps in discriminator
ndf = 64
ngpu=1

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

    def sample_latent(self, num_samples):
        return torch.randn((num_samples, 100, 1, 1))


class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

class Trainer():

    def __init__(self, generator, discriminator, gen_optimizer, dis_optimizer,
                 gp_weight=10, critic_iterations=5, print_every=500,
                 use_cuda=False, output_dir='./output'):
        self.G = generator
        self.G_opt = gen_optimizer
        self.D = discriminator
        self.D_opt = dis_optimizer
        self.losses = {'G': [], 'D': [], 'GP': [], 'gradient_norm': []}
        self.num_steps = 0
        self.use_cuda = use_cuda
        self.gp_weight = gp_weight
        self.critic_iterations = critic_iterations
        self.print_every = print_every
        self.output_dir = output_dir

        if self.use_cuda:
            self.G.cuda()
            self.D.cuda()

    def _critic_train_iteration(self, data):

        # Get generated data
        batch_size = data.size()[0]
        generated_data = self.sample_generator(batch_size)

        # Calculate probabilities on real and generated data
        data = Variable(data)
        if self.use_cuda:
            data = data.cuda()
        d_real = self.D(data)
        d_generated = self.D(generated_data)

        # Get gradient penalty
        gradient_penalty = self._gradient_penalty(data, generated_data)
        self.losses['GP'].append(gradient_penalty.item())

        # Create total loss and optimize
        self.D_opt.zero_grad()
        d_loss = d_generated.mean() - d_real.mean() + gradient_penalty
        d_loss.backward()
        self.D_opt.step()

        # Record loss
        self.losses['D'].append(d_loss.item())

    def _generator_train_iteration(self, data):

        self.G_opt.zero_grad()

        # Get generated data
        batch_size = data.size()[0]
        generated_data = self.sample_generator(batch_size)

        # Calculate loss and optimize
        d_generated = self.D(generated_data)
        g_loss = - d_generated.mean()
        g_loss.backward()
        self.G_opt.step()

        # Record loss
        self.losses['G'].append(g_loss.item())

    def _gradient_penalty(self, real_data, generated_data):
        batch_size = real_data.size()[0]

        # Calculate interpolation
        alpha = torch.rand(batch_size, 1, 1, 1)
        alpha = alpha.expand_as(real_data)
        if self.use_cuda:
            alpha = alpha.cuda()
        interpolated = alpha * real_data.data + (1 - alpha) * generated_data.data
        interpolated = Variable(interpolated, requires_grad=True)
        if self.use_cuda:
            interpolated = interpolated.cuda()

        # Calculate probability of interpolated examples
        prob_interpolated = self.D(interpolated)

        # Calculate gradients of probabilities with respect to examples
        gradients = torch_grad(outputs=prob_interpolated, inputs=interpolated,
                               grad_outputs=torch.ones(prob_interpolated.size()).cuda() if self.use_cuda else torch.ones(
                               prob_interpolated.size()),
                               create_graph=True, retain_graph=True)[0]

        # Gradients have shape (batch_size, num_channels, img_width, img_height), so flatten to easily take norm per example in batch
        gradients = gradients.view(batch_size, -1)
        self.losses['gradient_norm'].append(gradients.norm(2, dim=1).mean().item())

        # Derivatives of the gradient close to 0 can cause problems because of the square root, so manually calculate norm and add epsilon
        gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

        # Return gradient penalty
        return self.gp_weight * ((gradients_norm - 1) ** 2).mean()

    def _train_epoch(self, data_loader):
        for i, data in enumerate(data_loader):
            self.num_steps += 1
            self._critic_train_iteration(data[0])
            # Only update generator every |critic_iterations| iterations
            if self.num_steps % self.critic_iterations == 0:
                self._generator_train_iteration(data[0])

            if i % self.print_every == 0:
                print("Iteration {}".format(i + 1))
                print("D: {}".format(self.losses['D'][-1]))
                print("Gradient penalty: {}".format(self.losses['GP'][-1]))
                print("Gradient norm: {}".format(self.losses['gradient_norm'][-1]))
                if self.num_steps > self.critic_iterations:
                    print("G: {}".format(self.losses['G'][-1]))

    def train(self, data_loader, epochs, save_training_images=True):
        for epoch in range(epochs):
            print("\nEpoch {}".format(epoch + 1))
            self._train_epoch(data_loader)

            if save_training_images:
              self.save_training_images(epoch + 1)

            # if (epoch + 1) % 1 == 0:
            #     generator_path = f'./models_FP32_CIFAR10/generator_epoch_{epoch + 1}.pkl'
            #     discriminator_path = f'./models_FP32_CIFAR10/discriminator_epoch_{epoch + 1}.pkl'
            #     joblib.dump(self.G, generator_path)
            #     joblib.dump(self.D, discriminator_path)
            generator_path = f'./New_WGAN_checkpoints/generator_epoch_{epoch + 1}.pkl'
            discriminator_path = f'./New_WGAN_checkpoints/discriminator_epoch_{epoch + 1}.pkl'
            joblib.dump(self.G, generator_path)
            joblib.dump(self.D, discriminator_path)

    def sample_generator(self, num_samples):
        latent_samples = Variable(self.G.sample_latent(num_samples))
        if self.use_cuda:
            latent_samples = latent_samples.cuda()
        generated_data = self.G(latent_samples)
        return generated_data

    # def save_training_images(self, epoch):
    #     # Save batch of training images
    #     fixed_latents = Variable(self.G.sample_latent(64))
    #     if self.use_cuda:
    #         fixed_latents = fixed_latents.cuda()
    #     img_grid = make_grid(self.G(fixed_latents).cpu().data)
    #     img_grid = np.transpose(img_grid.numpy(), (1, 2, 0))
    #     imageio.imwrite('{}/training_images_epoch_{}.png'.format(self.output_dir, epoch), (img_grid * 255).astype(np.uint8))
    def save_training_images(self, epoch):
        # Save individual training images
        # Generate a batch of fixed random latent vectors using the generator's sample_latent method.
        num_img_to_generate = 10000
        fixed_latents = Variable(self.G.sample_latent(num_img_to_generate))

        # If GPU (CUDA) is used, move the latent vectors to the GPU.
        if self.use_cuda:
            fixed_latents = fixed_latents.cuda()

        # Specify the path of the folder you want to create
        folder_path = f'{self.output_dir}/{epoch}'

        # Check if the folder exists, and if not, create it
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Folder '{folder_path}' created successfully.")

        # Generate individual images from the fixed_latents using the generator (self.G).
        for i, latent in enumerate(fixed_latents):
            # Generate an image from the current latent vector.
            generated_image = self.G(latent.unsqueeze(0))  # Unsqueeze to add a batch dimension

            # Convert the generated image from GPU to CPU and get the data.
            generated_image_cpu = generated_image.cpu().data

            # Save the individual image as a PNG file with a filename that includes the epoch and index.
            imageio.imwrite(f'{folder_path}/training_image_epoch_{epoch}_{i}.png', (generated_image_cpu.squeeze().numpy() * 255).astype(np.uint8))


# Function to get data loaders for various datasets
def get_data_loaders(batch_size=128):

  transform = transforms.Compose([transforms.Resize(64), transforms.CenterCrop(64), transforms.ToTensor(), transforms.Normalize((0), (1)),])
  train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
  test_data = datasets.MNIST(root='./data', train=False, transform=transform)

  train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

  return train_loader, test_loader

# Device Configuration
start_time = time.time()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.backends.cudnn.enabled = False
print(device)

# # Create the "models" directory if it doesn't exist
# if not os.path.exists('./models_FP32_CIFAR10'):
#     os.makedirs('./models_FP32_CIFAR10')

# Initialize generator and discriminator
generator = Generator(ngpu).to(device)
generator.apply(weights_init)
discriminator = Discriminator(ngpu).to(device)
discriminator.apply(weights_init)

# Initialize optimizers
G_optimizer = torch.optim.Adam(generator.parameters(), lr=5e-4, betas=(.5, .999))
D_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-4, betas=(.5, .999))

# Set up trainer
output_directory = './New_WGAN_epochwise_images'
trainer = Trainer(generator, discriminator, G_optimizer, D_optimizer, use_cuda=torch.cuda.is_available(), output_dir=output_directory)

num_epoch = 10
data_loader, _ = get_data_loaders(batch_size=64)
trainer.train(data_loader, epochs=num_epoch, save_training_images=True)

print("Training Completed. It took: ", time.time() - start_time)

cuda

Epoch 1
Iteration 1
D: 3.461726188659668
Gradient penalty: 3.525785207748413
Gradient norm: 1.4071855545043945
Iteration 501
D: 0.5969372987747192
Gradient penalty: 1.0128111839294434
Gradient norm: 0.7579784989356995
G: -0.2523704767227173
Folder './New_WGAN_epochwise_images/1' created successfully.

Epoch 2
Iteration 1
D: -0.40593409538269043
Gradient penalty: 0.13966047763824463
Gradient norm: 0.9980427026748657
G: -0.3058271110057831
Iteration 501
D: -0.4012530446052551
Gradient penalty: 0.1361970156431198
Gradient norm: 0.9160897135734558
G: -0.3603123426437378
Folder './New_WGAN_epochwise_images/2' created successfully.

Epoch 3
Iteration 1
D: -0.04974338412284851
Gradient penalty: 0.21470561623573303
Gradient norm: 0.8883152008056641
G: -0.6096866726875305
Iteration 501
D: -0.19506779313087463
Gradient penalty: 0.0894557535648346
Gradient norm: 1.0058560371398926
G: -0.6397234201431274
Folder './New_WGAN_epochwise_images/3' created successfully.

Epoch 4
Iteration 1
D: 0.0

In [ ]:
%cd /content/

/content


In [ ]:
!zip -r output_new.zip output/

  adding: output/ (stored 0%)
  adding: output/training_images_epoch_16.png (deflated 2%)
  adding: output/training_images_epoch_2.png (deflated 2%)
  adding: output/training_images_epoch_9.png (deflated 2%)
  adding: output/training_images_epoch_4.png (deflated 3%)
  adding: output/training_images_epoch_11.png (deflated 2%)
  adding: output/training_images_epoch_15.png (deflated 1%)
  adding: output/training_images_epoch_18.png (deflated 1%)
  adding: output/training_images_epoch_1.png (deflated 2%)
  adding: output/training_images_epoch_7.png (deflated 3%)
  adding: output/training_images_epoch_12.png (deflated 2%)
  adding: output/training_images_epoch_14.png (deflated 1%)
  adding: output/training_images_epoch_17.png (deflated 2%)
  adding: output/training_images_epoch_10.png (deflated 2%)
  adding: output/training_images_epoch_8.png (deflated 3%)
  adding: output/training_images_epoch_6.png (deflated 3%)
  adding: output/training_images_epoch_19.png (deflated 1%)
  adding: output/

In [ ]:
from google.colab import files
files.download('output_new.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>